In [ ]:
from utils.helper import get_api_key
import dspy

In [ ]:
# Configure your values here
model_name = 'groq/llama-3.1-8b-instant'
api_key = get_api_key('GROQ_API_KEY')
api_endpoint = 'https://api.groq.com/openai/v1'

Configure the LLM with dspy, it uses liteLLM under the hood.

In [ ]:
llm = dspy.LM(
    model_name,
    api_key=api_key,
    api_base=api_endpoint,
)

# Set default LLM
dspy.settings.configure(lm=llm)

Let's define example data with a syntax error

In [ ]:
snippet = """
def build_query(table, columns, condition):
  query = "SELECT " + ", ".join(columns) +
          " FROM " + table +
          " WHERE " condition
  return query
"""
traceback = "invalid syntax (<unknown>, line 2)"

## Modules

First, we try to explain the problem with a simple prediction and compare it to a chain of thought approach.

In [ ]:
explain_module = dspy.Predict("snippet, context -> summary, confidence")
prediction = explain_module(snippet=snippet, context=traceback)

Checking **Predict** module and its history

In [ ]:
print("=== Predict ===")
print(explain_module)
print(prediction)

In [ ]:
dspy.inspect_history(n=1)

**Hint:**

Try to set type of confidence `str`, `int` or `float`. Like "summary, confidence: float"

In [ ]:
explain_module = dspy.ChainOfThought("snippet, context -> summary, confidence")
prediction = explain_module(snippet=snippet, context=traceback)

Checking **ChainOfThought** module and its history

In [ ]:
print("=== ChainOfThought ===")
print(explain_module)
print(prediction)

In [ ]:
dspy.inspect_history(n=1)

## Signature

**Signature** is a way to define the input and output of a module.

To add instructions, we need to define a signature first.

In [ ]:
inline_sign = dspy.Signature("snippet, context -> summary, confidence", instructions="Explain the problem in the code")
explain_with_inline = dspy.ChainOfThought(inline_sign)

In [ ]:
prediction_inline = explain_with_inline(snippet=snippet, context=traceback)
prediction_inline

In [ ]:
class ExplainSignature(dspy.Signature):
    """Explain the problem in the code"""
    snippet = dspy.InputField()
    context = dspy.InputField()
    reasoning = dspy.OutputField()
    summary = dspy.OutputField()
    confidence: float = dspy.OutputField()

ExplainSignature

In [ ]:
explain_with_class = dspy.ChainOfThought(ExplainSignature)
prediction_with_class = explain_with_class(snippet=snippet, context=traceback)
prediction_with_class

Task:

- Extend signature by adding `description` to each field, like `description = "Snippet of code"`
- Possible descriptions:
    - `snippet`: "The code snippet that caused the error"
    - `context`: "Extra details about the error"
    - `reasoning`: "Think deeply to achieve max confidence"
    - `summary`: "Issue explanation. Provide a detailed explanation of the issue. To make it easy to understand, use Markdown syntax."
    - `confidence`: "Confidence score from 0 to 1 indicating the certainty of the explanation"

Validate what happen with confidence and summary quality.

In [ ]:
# ========================================
# TASK: Build Your Own Signature
# ========================================
# Create a signature class with detailed descriptions
#
# TODO: Complete the signature below
# Add ALL the suggested descriptions
# ========================================

class ExplainSignature(dspy.Signature):
    """Explain the problem in the code"""

    # YOUR CODE HERE:
    # Add these fields with descriptions:
    # snippet: "The code snippet that caused the error"
    # context: "Extra details about the error"
    # reasoning: "Think deeply to achieve max confidence"
    # summary: "Issue explanation. Provide a detailed explanation of the issue. To make it easy to understand, use Markdown syntax."
    # confidence: float - "Confidence score from 0 to 1 indicating the certainty of the explanation"

    snippet = dspy.InputField(description="...")  # TODO: Add description
    context = dspy.InputField(description="...")  # TODO: Add description
    reasoning = dspy.OutputField(description="...") # TODO: Add description
    summary = dspy.OutputField(description="...")  # TODO: Add description
    confidence: float = dspy.OutputField(description="...")  # TODO: Add description

# Test your signature
# explain_custom = dspy.ChainOfThought(ExplainSignature)
# result = explain_custom(snippet=snippet, context=traceback)
# print(result.summary)
# print("Confidence:", result.confidence)

Compare no signature, inline signature and class signature results.